<a href="https://colab.research.google.com/github/RedPanda54/Machine_Learning_study/blob/main/kNN_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(0) 'pandas'를 이용해 'fake_bills' csv파일을 불러옴.

In [ ]:
import pandas as pd
target_URL = "https://raw.githubusercontent.com/RedPanda54/Machine_Learning_study/main/fake_bills.csv"
df = pd.read_csv(target_URL, sep=';')
df.info() # information에서 확인해보면 'margin_low'열에 null 자리가 37개가 있음을 알 수 있다.

In [ ]:
df.isna().sum() # 좀 더 명확하게 결측치 총 개수를 확인할 수 있다.

(1) One_hot_encoding

In [ ]:
one_hot_encoding = pd.get_dummies(df, columns = ['margin_low'], dummy_na = True)
print(one_hot_encoding)

(2)-i 'missingno'를 사용해 결측치 시각화

In [ ]:
import missingno as msno
msno.matrix(df, figsize = (12, 5)) # matrix 키워드로 결측치가 발생한 대략적인 위치 파악

In [ ]:
msno.bar(df, figsize = (12, 5)) # bar 키워드로 수치까지 표현해서 몇 개의 결측치가 있는지 파악

(2)-ii kNN을 이용해 결측치 채우기

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

missing_values = imputer.missing_values
'''
for col, idx in zip(np.where(df.isna())[1], np.where(df.isna())[0]):
    if col == np.where(df.isna())[1][0]:
        print("index %d, column %s" % (idx, df.columns[col]))
        _, neighbors = imputer.kneighbors([df_imputed.iloc[idx, :]])
        print("Neighbors: ", neighbors[0][1:])
        print("Original value: ", df.iloc[idx, col])
        print("Imputed value: ", df_imputed.iloc[idx, col])
'''
filled_df = imputer.fit_transform(df)
pd.DataFrame(filled_df)
# 데이터프레임을 확인해보면 4번 column에 무한소수가 껴있는것이 보일것이다. 결측치를 kNNImputer로 채운 결과이다.

In [ ]:
print(type(df))
print(type(filled_df)) # 원래 dataset과 결측치를 보간한 dataset은 데이터 타입이 다르다.

(3) 자료 확인 (np.corrcoef)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr_mat = np.corrcoef(filled_df,rowvar=False) # 상관계수를 계산해주는 함수
print(np.round(corr_mat,3)) # 완전히 symmetric한 형태

In [ ]:
corr_df = pd.DataFrame(corr_mat,columns=df.columns,index=df.columns)
sns.heatmap(corr_df, linewidth=1, vmax = 1.0, cmap='plasma')
plt.figure(figsize=(10, 8))
plt.show() # heatmap으로 표현

(4) standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
data_standadized = StandardScaler().fit_transform(filled_df)
pd.DataFrame(data_standadized)

(5) train dataset, test dataset 구분

In [ ]:
df_data = pd.DataFrame(data_standadized)
#df_labels = pd.DataFrame(data_standadized)
feature_df = df_data.iloc[:, :-1]
target_df = df_data.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

training_data, test_data , training_labels, test_labels = train_test_split(feature_df, target_df, test_size = 0.3, stratify = target_df, random_state = 100)

In [ ]:
from sklearn.model_selection import train_test_split

training_data, test_data , training_labels, test_labels = train_test_split(feature_df, target_df, test_size = 0.3, random_state = 100)

In [ ]:
print(len(training_data))
print(len(test_data))
print(len(training_labels))
print(len(test_labels))

(6) GridSearch

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 25)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid = param_grid, cv=5)
grid.fit(training_data, training_labels)